In [1]:
import pandas as pd
from utils.sentiment import get_sentiment_score
from utils.indicators import add_technical_indicators

In [18]:
# Load datasets

prices = pd.read_csv("../data/raw/stock_prices.csv")
tickers = pd.read_csv("../data/raw/full_ticker_names.csv")

print("Prices shape:", prices.shape)
print("Tickers shape:", tickers.shape)
print(tickers.head())

Prices shape: (2720, 25)
Tickers shape: (11133, 2)
  Symbol                                      Security Name
0   AACG  ATA Creativity Global - American Depositary Sh...
1   AADI               Aadi Bioscience, Inc. - Common Stock
2   AADR                AdvisorShares Dorsey Wright ADR ETF
3    AAL       American Airlines Group, Inc. - Common Stock
4   AAME       Atlantic American Corporation - Common Stock


In [19]:
prices = pd.read_csv("../data/raw/stock_prices.csv", skiprows=2)
print("First few rows:")
print(prices.head())

First few rows:
         Date  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0  2014-01-02   37.160000   27.855856   54.709999     0.39650   58.209999   
1  2014-01-03   36.910000   27.652653   54.560001     0.39175   58.660000   
2  2014-01-06   36.130001   27.960960   57.200001     0.39700   59.000000   
3  2014-01-07   36.410000   28.500000   57.919998     0.40350   58.320000   
4  2014-01-08   35.759998   28.559309   58.230000     0.40900   58.869999   

   Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  Unnamed: 15  \
0       16.10   83.414001   55.252499   91.029999  ...    38.584999   
1       16.41   83.081001   55.290001   91.849998  ...    38.474998   
2       16.66   82.668999   54.957500   92.330002  ...    38.084999   
3       16.50   83.709000   55.377499   94.290001  ...    38.605000   
4       16.58   83.683998   55.555000   94.160004  ...    39.014999   

   Unnamed: 16  Unnamed: 17  Unnamed: 18  Unnamed: 19  Unnamed: 20  \
0    68.940002          

In [17]:
print(prices.iloc[:2, :8])

         Date  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0  2014-01-02       37.16   27.855856   54.709999     0.39650   58.209999   
1  2014-01-03       36.91   27.652653   54.560001     0.39175   58.660000   

   Unnamed: 6  Unnamed: 7  
0       16.10   83.414001  
1       16.41   83.081001  


In [22]:
# Select the first 24 stock symbols
symbols = tickers['Symbol'].head(24).tolist()
print(symbols)

['AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAXJ', 'ABAT', 'ABCL', 'ABCS', 'ABEO', 'ABL', 'ABLLL', 'ABLLW', 'ABLV', 'ABLVW', 'ABNB', 'ABOS', 'ABSI']


In [23]:
#Create the mapping
col_map = {'Unnamed: 0': 'Date'}

# Loop through symbols to rename columns
for i, symbol in enumerate(symbols, start=1):
    col_map[f'Unnamed: {i}'] = symbol

# Apply renaming
prices.rename(columns=col_map, inplace=True)

print("Renamed columns:")
print(prices.columns.tolist())

Renamed columns:
['Date', 'AACG', 'AADI', 'AADR', 'AAL', 'AAME', 'AAOI', 'AAON', 'AAPB', 'AAPD', 'AAPL', 'AAPU', 'AAXJ', 'ABAT', 'ABCL', 'ABCS', 'ABEO', 'ABL', 'ABLLL', 'ABLLW', 'ABLV', 'ABLVW', 'ABNB', 'ABOS', 'ABSI']


In [24]:
# Reshape data to long format
long_prices = prices.melt(
    id_vars=["Date"],
    var_name="Ticker",
    value_name="Close"
)

# Convert dates and drop missing
long_prices["Date"] = pd.to_datetime(long_prices["Date"], errors="coerce")
long_prices.dropna(subset=["Date", "Close"], inplace=True)

print("Long-format shape:", long_prices.shape)
long_prices.head()


Long-format shape: (59533, 3)


,Date,Ticker,Close
0,2014-01-02,AACG,37.160000
1,2014-01-03,AACG,36.910000
2,2014-01-06,AACG,36.130001
3,2014-01-07,AACG,36.410000
4,2014-01-08,AACG,35.759998


In [27]:
#Save clean dataset
long_prices.to_csv("../data/processed/clean_stock_prices.csv", index=False)
print("Saved clean dataset to ../data/raw/clean_stock_prices.csv")

Saved clean dataset to ../data/raw/clean_stock_prices.csv


In [28]:
print(long_prices['Ticker'].unique()[:10])  # first 10 tickers
print(long_prices.describe())

['AACG' 'AADI' 'AADR' 'AAL' 'AAME' 'AAOI' 'AAON' 'AAPB' 'AAPD' 'AAPL']
                                Date         Close
count                          59533  59533.000000
mean   2019-07-22 09:48:32.932323072    105.493065
min              2014-01-02 00:00:00      0.384000
25%              2016-10-24 00:00:00     40.567501
50%              2019-08-20 00:00:00     78.309998
75%              2022-05-02 00:00:00    132.229996
max              2024-10-18 00:00:00    595.940002
std                              NaN     94.158803


In [29]:
#Extract sentiment scores from financial news dataset
news = pd.read_csv("../data/raw/raw_partner_headlines.csv")

print("News shape:", news.shape)
print("Columns:", news.columns.tolist())
news.head(3)

News shape: (1845559, 6)
Columns: ['Unnamed: 0', 'headline', 'url', 'publisher', 'date', 'stock']


,Unnamed: 0,headline,url,publisher,date,stock
0,2,Agilent Technologies Announces Pricing of $5……...,http://www.gurufocus.com/news/1153187/agilent-...,GuruFocus,2020-06-01 00:00:00,A
1,3,Agilent (A) Gears Up for Q2 Earnings: What's i...,http://www.zacks.com/stock/news/931205/agilent...,Zacks,2020-05-18 00:00:00,A
2,4,J.P. Morgan Asset Management Announces Liquida...,http://www.gurufocus.com/news/1138923/jp-morga...,GuruFocus,2020-05-15 00:00:00,A


In [31]:
# Keep only the necessary columns
news = news[['stock', 'date', 'headline']]

# Drop missing values
news = news.dropna(subset=['stock', 'date', 'headline'])

# Convert date to proper datetime format
news['date'] = pd.to_datetime(news['date'], errors='coerce')

# Remove rows with invalid dates
news = news.dropna(subset=['date'])

# Standardize column names
news.rename(columns={'stock': 'Ticker'}, inplace=True)

print("Cleaned news shape:", news.shape)
print(news.head(3))

Cleaned news shape: (1845559, 3)
  Ticker       date                                           headline
0      A 2020-06-01  Agilent Technologies Announces Pricing of $5……...
1      A 2020-05-18  Agilent (A) Gears Up for Q2 Earnings: What's i...
2      A 2020-05-15  J.P. Morgan Asset Management Announces Liquida...


In [32]:
#Compute sentiment scores
from utils.sentiment import get_sentiment_score

print("Calculating sentiment scores (this may take a few minutes)...")

news["sentiment"] = news["headline"].apply(get_sentiment_score)

print("Example results:")
print(news.head(5)[["Ticker", "headline", "sentiment"]])

Calculating sentiment scores (this may take a few minutes)...
Example results:
  Ticker                                           headline  sentiment
0      A  Agilent Technologies Announces Pricing of $5……...        0.0
1      A  Agilent (A) Gears Up for Q2 Earnings: What's i...        0.0
2      A  J.P. Morgan Asset Management Announces Liquida...        0.0
3      A  Pershing Square Capital Management, L.P. Buys ...        0.0
4      A  Agilent Awards Trilogy Sciences with a Golden ...        0.3


In [43]:
#Aggregate Sentiment by Date and Ticker
daily_sentiment = (
    news.groupby(["Ticker", "date"])["sentiment"]
    .mean()
    .reset_index()
)

print("Daily sentiment sample:")
print(daily_sentiment.head(10))

✅ Daily sentiment sample:
  Ticker       date  sentiment
0      A 2012-05-14   0.000000
1      A 2012-05-15   0.000000
2      A 2012-05-16   0.150000
3      A 2012-05-17   0.022727
4      A 2012-05-22   0.000000
5      A 2012-05-30   0.136364
6      A 2012-06-04   0.000000
7      A 2012-06-08   0.000000
8      A 2012-06-15   0.068182
9      A 2012-06-19   0.000000


In [44]:
# Remove invalid or default epoch dates
daily_sentiment = daily_sentiment[daily_sentiment["date"] > "1970-01-01"]

print("🧹 Cleaned news date range:",
      daily_sentiment["date"].min(), "→",
      daily_sentiment["date"].max())

🧹 Cleaned news date range: 2010-02-03 00:00:00 → 2020-06-04 00:00:00


In [45]:
#Merge on Ticker + Date
# Identify overlapping tickers
price_tickers = set(prices["Ticker"].unique())
news_tickers = set(daily_sentiment["Ticker"].unique())
overlap_tickers = price_tickers.intersection(news_tickers)

print("Number of tickers in prices:", len(price_tickers))
print("Number of tickers in news:", len(news_tickers))
print("Overlapping tickers:", len(overlap_tickers))
print("Example overlaps:", list(overlap_tickers)[:10])

# Filter prices for overlapping tickers and valid date range
min_date = daily_sentiment["date"].min()
max_date = daily_sentiment["date"].max()

filtered_prices = prices[
    (prices["Ticker"].isin(overlap_tickers)) &
    (prices["Date"] >= min_date) &
    (prices["Date"] <= max_date)
    ]

print("Filtered prices shape:", filtered_prices.shape)
filtered_prices.head()


Number of tickers in prices: 24
Number of tickers in news: 6552
Overlapping tickers: 7
Example overlaps: ['AAXJ', 'AAPL', 'AAME', 'AAON', 'AADR', 'AAL', 'AAOI']
Filtered prices shape: (11319, 3)


,Date,Ticker,Close
5436,2014-01-02,AADR,54.709999
5437,2014-01-03,AADR,54.560001
5438,2014-01-06,AADR,57.200001
5439,2014-01-07,AADR,57.919998
5440,2014-01-08,AADR,58.230000


In [46]:
#Merge prices + sentiment
merged = pd.merge(
    filtered_prices,
    daily_sentiment,
    left_on=["Ticker", "Date"],
    right_on=["Ticker", "date"],
    how="left"
)

# Drop redundant column
merged.drop(columns=["date"], inplace=True)

print("Merged dataset shape:", merged.shape)
print("Sample merged rows:")
print(merged.head(10))


Merged dataset shape: (11319, 4)
Sample merged rows:
        Date Ticker      Close  sentiment
0 2014-01-02   AADR  54.709999        NaN
1 2014-01-03   AADR  54.560001        NaN
2 2014-01-06   AADR  57.200001        NaN
3 2014-01-07   AADR  57.919998        NaN
4 2014-01-08   AADR  58.230000        NaN
5 2014-01-09   AADR  57.220001        NaN
6 2014-01-10   AADR  57.939999        NaN
7 2014-01-13   AADR  55.910000        NaN
8 2014-01-14   AADR  57.740002        NaN
9 2014-01-15   AADR  57.599998        NaN


In [47]:
print("Tickers with sentiment coverage:")
print(merged.loc[merged["sentiment"].notna(), "Ticker"].unique())

Tickers with sentiment coverage:
['AADR' 'AAL' 'AAME' 'AAOI' 'AAON' 'AAPL' 'AAXJ']


In [50]:
def compute_rsi(series: pd.Series, period: int = 14):
    """Relative Strength Index (RSI)"""
    delta = series.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    return 100 - (100 / (1 + rs))

def compute_macd(series: pd.Series, short=12, long=26):
    """MACD: difference between short and long EMA"""
    ema_short = series.ewm(span=short, adjust=False).mean()
    ema_long = series.ewm(span=long, adjust=False).mean()
    return ema_short - ema_long

def add_technical_indicators(df: pd.DataFrame):
    """
    Add RSI, MACD, EMA(10), EMA(30), and EMA ratio for each ticker.
    """

    df = df.copy()

    # Ensure data is sorted
    df.sort_values(by=["Ticker", "Date"], inplace=True)

    indicators = []
    for ticker, group in df.groupby("Ticker"):
        group = group.copy()

        group["rsi"] = compute_rsi(group["Close"])
        group["macd"] = compute_macd(group["Close"])
        group["ema_10"] = group["Close"].ewm(span=10, adjust=False).mean()
        group["ema_30"] = group["Close"].ewm(span=30, adjust=False).mean()
        group["ema_ratio"] = group["ema_10"] / group["ema_30"]

        indicators.append(group)

    enriched_df = pd.concat(indicators, ignore_index=True)

    # Fill NaNs (first few rows of indicators) with 0
    enriched_df.fillna(0, inplace=True)

    return enriched_df

In [53]:
def compute_rsi(prices: pd.Series, period: int = 14):
    """Relative Strength Index"""
    prices = pd.to_numeric(prices, errors="coerce")
    delta = prices.diff()
    gain = delta.clip(lower=0)
    loss = -delta.clip(upper=0)
    avg_gain = gain.rolling(window=period, min_periods=period).mean()
    avg_loss = loss.rolling(window=period, min_periods=period).mean()
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi.fillna(0)


def compute_macd(prices: pd.Series, short=12, long=26):
    """MACD (Moving Average Convergence Divergence)"""
    prices = pd.to_numeric(prices, errors="coerce")
    ema_short = prices.ewm(span=short, adjust=False).mean()
    ema_long = prices.ewm(span=long, adjust=False).mean()
    macd = ema_short - ema_long
    return macd.fillna(0)


def add_technical_indicators(df: pd.DataFrame):
    """Add RSI, MACD, EMA10, EMA30, EMA ratio per ticker"""
    df = df.copy()
    df["Close"] = pd.to_numeric(df["Close"], errors="coerce")
    df.sort_values(["Ticker", "Date"], inplace=True)

    result = []
    for ticker, group in df.groupby("Ticker"):
        group = group.copy()
        group["rsi"] = compute_rsi(group["Close"])
        group["macd"] = compute_macd(group["Close"])
        group["ema_10"] = group["Close"].ewm(span=10, adjust=False).mean()
        group["ema_30"] = group["Close"].ewm(span=30, adjust=False).mean()
        group["ema_ratio"] = group["ema_10"] / group["ema_30"]
        result.append(group)

    enriched = pd.concat(result, ignore_index=True)
    enriched.fillna(0, inplace=True)
    return enriched


# Apply indicator generation
print("Computing technical indicators...")
merged["Close"] = pd.to_numeric(merged["Close"], errors="coerce")
enriched = add_technical_indicators(merged)

print("Indicators added:")
print([col for col in enriched.columns if col not in ["Date", "Ticker", "Close", "sentiment"]])
print(enriched.head(10))

Computing technical indicators...
Indicators added:
['rsi', 'macd', 'ema_10', 'ema_30', 'ema_ratio']
        Date Ticker      Close  sentiment  rsi      macd     ema_10  \
0 2014-01-02   AADR  54.709999        0.0  0.0  0.000000  54.709999   
1 2014-01-03   AADR  54.560001        0.0  0.0 -0.011966  54.682727   
2 2014-01-06   AADR  57.200001        0.0  0.0  0.189394  55.140413   
3 2014-01-07   AADR  57.919998        0.0  0.0  0.402432  55.645792   
4 2014-01-08   AADR  58.230000        0.0  0.0  0.589485  56.115648   
5 2014-01-09   AADR  57.220001        0.0  0.0  0.648750  56.316439   
6 2014-01-10   AADR  57.939999        0.0  0.0  0.745224  56.611632   
7 2014-01-13   AADR  55.910000        0.0  0.0  0.650380  56.484063   
8 2014-01-14   AADR  57.740002        0.0  0.0  0.714643  56.712415   
9 2014-01-15   AADR  57.599998        0.0  0.0  0.745680  56.873794   

      ema_30  ema_ratio  
0  54.709999   1.000000  
1  54.700322   0.999678  
2  54.861591   1.005082  
3  55.058908 

In [54]:
enriched.to_csv("../data/processed/final_dataset.csv", index=False)
print("Saved final enriched dataset to ../data/processed/final_dataset.csv")

Saved final enriched dataset to ../data/processed/final_dataset.csv
